# Bank Totals Database Creation
## Created by Eryn Carleton
Last modified 9/8/2023

Objectives:
 1. Read census data and national HMDA LAR
 2. Join the data
 3. Calculate various totals
 4. Create table with totals in SQLite


In [1]:
import csv
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from scipy.stats import fisher_exact
import sqlite3

In [3]:
'''
con = sqlite3.connect('hmda_database')

cur = con.cursor()

#delete the existing table

query = 'DROP TABLE hmda_totals'
con.execute(query)
con.commit()
con.close()
'''

"\ncon = sqlite3.connect('hmda_database')\n\ncur = con.cursor()\n\n#delete the existing table\n\nquery = 'DROP TABLE hmda_totals'\ncon.execute(query)\ncon.commit()\ncon.close()\n"

In [4]:
#read in 2022 census data

census = pd.read_csv('CensusFlatFile2022.csv', low_memory = False)

len(census.columns)

1212

In [5]:
#census_named = census.iloc[:,[1,2,3,4,5,23,28,29,31,32,64,65,66,67,68,69,70,13,12,1212,23,24]]
census = census.iloc[:,[0,1,2,3,4,22,27,28,30,31,63,64,65,66,67,68,69,12,11,1211,22,23]]

census.columns = ['year','msa_number','fips_state_cd','fips_county_cd','census_tract','pop','minority_pop','minority_pct',
                  'hispanic_pop','non_hispanic_pop','white_pop','black_pop','ai_pop','asian_pop','pi_pop','other_pop',
                  'two_or_more_pop','tract_income_as_pct','msa_median_hh_income','median_oo_home_value','nbr_families',
                  'nbr_hh']

census.head()

,year,msa_number,fips_state_cd,fips_county_cd,census_tract,pop,minority_pop,minority_pct,hispanic_pop,non_hispanic_pop,...,ai_pop,asian_pop,pi_pop,other_pop,two_or_more_pop,tract_income_as_pct,msa_median_hh_income,median_oo_home_value,nbr_families,nbr_hh
0,2022,33860,1,1,20200,2055.0,1221.0,59.42,43.0,2012.0,...,2.0,12.0,4.0,20.0,71.0,73.60,54250.0,85700.0,2055.0,359.0
1,2022,33860,1,1,20300,3216.0,996.0,30.97,100.0,3116.0,...,12.0,14.0,6.0,49.0,177.0,102.93,54250.0,115000.0,3216.0,975.0
2,2022,33860,1,1,20400,4246.0,724.0,17.05,158.0,4088.0,...,17.0,32.0,1.0,38.0,253.0,110.95,54250.0,163100.0,4246.0,1137.0
3,2022,33860,1,1,20501,4322.0,1111.0,25.71,156.0,4166.0,...,29.0,93.0,2.0,58.0,267.0,133.41,54250.0,173000.0,4322.0,1278.0
4,2022,33860,1,1,20502,3284.0,1433.0,43.64,165.0,3119.0,...,15.0,207.0,3.0,66.0,192.0,115.06,54250.0,186800.0,3284.0,845.0


In [6]:
#correct datatypes and add leading zeros where applicable

c = census

c[['msa_number','fips_state_cd','fips_county_cd','census_tract']] = c[['msa_number','fips_state_cd','fips_county_cd','census_tract']].astype(str)

c['msa_number'] = c['msa_number'].str.zfill(5)
c['fips_state_cd'] = c['fips_state_cd'].str.zfill(2)
c['fips_county_cd'] = c['fips_county_cd'].str.zfill(3)
c['census_tract'] = c['census_tract'].str.zfill(6)

c['tract'] = c['fips_state_cd'] + c['fips_county_cd'] + c['census_tract']

c['mmt_flag'] = np.where(c['minority_pct'] >= 50, 1, 0)
c['hmt_flag'] = np.where(c['minority_pct'] >= 80, 1, 0)
c['lmi_flag'] = np.where(c['tract_income_as_pct'] <= 80, 1, 0)

conditions = [(c['tract_income_as_pct'] <= 50),
              (c['tract_income_as_pct'] > 50) & (c['tract_income_as_pct'] <= 80),
              (c['tract_income_as_pct'] > 80) & (c['tract_income_as_pct'] <= 120),
              (c['tract_income_as_pct'] > 120)]

values = ['Low','Moderate','Middle','Upper']

c['income_level'] = np.select(conditions, values)

c['maj_hispanic'] = np.where(c['hispanic_pop']/c['pop'] >= 0.5, 1, 0)
c['maj_black'] = np.where(c['black_pop']/c['pop'] >= 0.5, 1, 0)
c['maj_asian'] = np.where(c['asian_pop']/c['pop'] >= 0.5, 1, 0)
c['maj_ai'] = np.where(c['ai_pop']/c['pop'] >= 0.5, 1, 0)
c['maj_pi'] = np.where(c['pi_pop']/c['pop'] >= 0.5, 1, 0)

c.head()

len(c)

87274

In [7]:
#read in msa data

msas = pd.read_csv('MSAs.csv', encoding = "ISO-8859-1")

msas = msas.iloc[:,[0,3,4,5,6]].drop_duplicates()

#join to census data

census_final = c.set_index('msa_number').join(msas.set_index('CBSA Code'), how = 'left')

#census_final['Metropolitan/Micropolitan Statistical Area'].unique()

In [8]:
#there are too many damn cities, so we're going to filter it down to cities of 1M or cities of 50k in the Zions footprint

size_thresh = pd.pivot_table(census_final,
                             values = ['pop'],
                             index = 'CBSA Title',
                             aggfunc = np.sum)

#the pivot uses the bank_name columns as a label rather than column name
        #make it be a column name

size_thresh.reset_index(inplace = True)

size_thresh.rename(columns = {'pop': 'total_pop'}, inplace = True)

census_final = census_final.join(size_thresh.set_index('CBSA Title'), on = 'CBSA Title', how = 'inner', lsuffix = '', rsuffix = '.y')

size_thresh
'''
census_final = census_final.loc[(census_final['total_pop'] >= 1000000) |
                 ((census_final['fips_state_cd'] == '04')|
                 (census_final['fips_state_cd'] == '06')|
                 (census_final['fips_state_cd'] == '08')|
                 (census_final['fips_state_cd'] == '16')|
                 (census_final['fips_state_cd'] == '32')|
                 (census_final['fips_state_cd'] == '35')|
                 (census_final['fips_state_cd'] == '41')|
                 (census_final['fips_state_cd'] == '48')|
                 (census_final['fips_state_cd'] == '49')|
                 (census_final['fips_state_cd'] == '53')|
                 (census_final['fips_state_cd'] == '56') & census_final['total_pop'] >= 50000)]
'''

census_final = census_final.loc[census_final['total_pop'] >= 1000000]

len(census_final['CBSA Title'].unique())

47

In [9]:
#define the function that puts the each msa's data into the database

def msa_table_maker(your_msa):
    
    if type(i) == float:
        pass
    else:

        con = sqlite3.connect('hmda_database')

        cur = con.cursor()

        msa = your_msa

        print(msa)

        msa_tracts = census_final.loc[census_final['CBSA Title'] == msa]

        msa_tracts = msa_tracts['tract'].values.tolist()

        msa_tracts = pd.to_numeric(msa_tracts)

        lar = pd.DataFrame()

        for chunk in pd.read_csv('2022_combined_mlar_header.txt', sep = '|', iterator = True, chunksize = 10000, low_memory = False):
            temp = chunk.query('census_tract in @msa_tracts')
            if len(temp) > 0:
                #lar = lar.append(temp)
                lar = pd.concat([lar, temp])
            else:
                pass

        #clean up LAR's tract field

        lar['tract'] = lar['census_tract'].astype(str).str[0:11]

        #join lar to census data

        lar = lar.join(census_final.set_index('tract'), on = 'tract', lsuffix = '', rsuffix = '_c')

        #read in names of banks

        panel = pd.read_csv('reporter panel.csv')

        panel = panel.iloc[:,[0,1,6,9,10,12,14]]

        panel.head()

        #join the lar to the panel to get add the banks to file

        lar_banks = lar.join(panel.set_index('lei'), on = 'lei', lsuffix = '', rsuffix = '_panel')

        lar_banks['bank_name'] = lar_banks.respondent_name.combine_first(lar_banks.lei)

        #filter to only depository institutions

        lar_banks = lar_banks.query('other_lender_code == 0')

        #filter to only non-exempt

        lar_banks = lar_banks.query('open_end_line_of_credit != 1111')

        #join to census data

        lar_census = lar_banks.join(panel.set_index('lei'), on = 'lei', lsuffix = '', rsuffix = '_panel')

        #create counter fields

        lar_banks['bank_count'] = 1
        lar_banks['bank_count_mmt'] = np.where(lar_banks['mmt_flag'] == 1, 1, 0)
        lar_banks['bank_count_lmi'] = np.where(lar_banks['lmi_flag'] == 1, 1, 0)
        lar_banks['heloc'] = np.where(lar_banks['open_end_line_of_credit'] == 1, 1, 0)
        lar_banks['heloc_mmt'] = np.where(lar_banks['mmt_flag'] + lar_banks['heloc'] == 2, 1, 0)
        lar_banks['heloc_lmi'] = np.where(lar_banks['lmi_flag'] + lar_banks['heloc'] == 2, 1, 0)
        lar_banks['originated'] = np.where(lar_banks['action_taken'] == 1, 1, 0)
        lar_banks['originated_mmt'] = np.where(lar_banks['mmt_flag'] + lar_banks['originated'] == 2,1,0)
        lar_banks['originated_lmi'] = np.where(lar_banks['lmi_flag'] + lar_banks['originated'] == 2,1,0)
        lar_banks['originated_heloc'] = np.where(lar_banks['heloc'] + lar_banks['originated'] == 2,1,0)
        lar_banks['originated_heloc_mmt'] = np.where(lar_banks['mmt_flag'] + lar_banks['originated_heloc'] == 2, 1, 0)
        lar_banks['originated_heloc_lmi'] = np.where(lar_banks['mmt_flag'] + lar_banks['originated_heloc'] == 2, 1, 0)


        #make pivot

        bank_totals = pd.pivot_table(lar_banks,
                                     values = ['bank_count','bank_count_mmt','bank_count_lmi',
                                               'heloc','heloc_mmt','heloc_lmi',
                                               'originated','originated_mmt','originated_lmi',
                                               'originated_heloc','originated_heloc_mmt','originated_heloc_lmi'],
                                     index = 'bank_name',
                                     aggfunc = np.sum)

        #the pivot uses the bank_name columns as a label rather than column name
        #make it be a column name

        bank_totals.reset_index(inplace = True)

        #create calculated fields

        bank_totals['low_thresh'] = bank_totals['bank_count'] * .5
        bank_totals['high_thresh'] = bank_totals['bank_count'] * 2
        bank_totals['low_heloc_thresh'] = bank_totals['heloc'] * .5
        bank_totals['high_heloc_thresh'] = bank_totals['heloc'] * 2
        bank_totals['pct_mmt'] = bank_totals['bank_count_mmt']/bank_totals['bank_count']
        bank_totals['pct_lmi'] = bank_totals['bank_count_lmi']/bank_totals['bank_count']
        bank_totals['heloc_pct_mmt'] = bank_totals['heloc_mmt']/bank_totals['heloc']
        bank_totals['heloc_pct_lmi'] = bank_totals['heloc_lmi']/bank_totals['heloc']
        bank_totals['originated_pct_mmt'] = bank_totals['originated_mmt']/bank_totals['originated']
        bank_totals['originated_pct_lmi'] = bank_totals['originated_lmi']/bank_totals['originated']
        bank_totals['originated_heloc_pct_mmt'] = bank_totals['originated_heloc_mmt']/bank_totals['originated_heloc']
        bank_totals['originated_heloc_pct_lmi'] = bank_totals['originated_heloc_lmi']/bank_totals['originated_heloc']
        bank_totals['msa'] = msa

        #filter to only banks with at least 10 applications in the MSA

        bank_totals = bank_totals.query('bank_count >= 10')

        #put the table in the database

        bank_totals.to_sql('hmda_totals', con, if_exists = 'append')

        rows = pd.read_sql('select COUNT(*) from hmda_totals', con)

        print(rows)

        con.close()


In [10]:
for i in census_final['CBSA Title'].unique():
    msa_table_maker(i)

Atlanta-Sandy Springs-Alpharetta, GA
   COUNT(*)
0       194
Austin-Round Rock-Georgetown, TX
   COUNT(*)
0       336
Baltimore-Columbia-Towson, MD
   COUNT(*)
0       468
Birmingham-Hoover, AL
   COUNT(*)
0       554
Buffalo-Cheektowaga, NY
   COUNT(*)
0       610
Charlotte-Concord-Gastonia, NC-SC
   COUNT(*)
0       759
Cincinnati, OH-KY-IN
   COUNT(*)
0       881
Cleveland-Elyria, OH
   COUNT(*)
0       972
Columbus, OH
   COUNT(*)
0      1095
Denver-Aurora-Lakewood, CO
   COUNT(*)
0      1250
Fresno, CA
   COUNT(*)
0      1313
Grand Rapids-Kentwood, MI
   COUNT(*)
0      1397
Hartford-East Hartford-Middletown, CT
   COUNT(*)
0      1497
Houston-The Woodlands-Sugar Land, TX
   COUNT(*)
0      1696
Indianapolis-Carmel-Anderson, IN
   COUNT(*)
0      1818
Jacksonville, FL
   COUNT(*)
0      1936
Kansas City, MO-KS
   COUNT(*)
0      2066
Las Vegas-Henderson-Paradise, NV
   COUNT(*)
0      2185
Louisville/Jefferson County, KY-IN
   COUNT(*)
0      2265
Memphis, TN-MS-AR
   COUNT(*)
0  

In [11]:
#check that the table was created correctly

con = sqlite3.connect('hmda_database')

query = 'select * from hmda_totals;'

res = pd.read_sql(query, con)

con.close()

print(res)

      index                               bank_name  bank_count  \
0         7                                AFFINITY          45   
1         9                                  ALL IN          21   
2        10           ALLEGACY FEDERAL CREDIT UNION         131   
3        12                    ALLIANT CREDIT UNION         871   
4        16            ANDREWS FEDERAL CREDIT UNION          78   
...     ...                                     ...         ...   
5342    263                             United Bank          33   
5343    269             Virginia Credit Union, Inc.         135   
5344    274  Wells Fargo Bank, National Association        3144   
5345    276                  West Town Bank & Trust          10   
5346    278    Wilmington Savings Fund Society, FSB          32   

      bank_count_lmi  bank_count_mmt  heloc  heloc_lmi  heloc_mmt  originated  \
0                  5              22     30          3         16          21   
1                  7             